# Gathering Data

In [8]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import html
import json

In [11]:
url = r'https://www.rev.com/blog/transcripts/we-have-evidence-of-russian-war-crimes-says-ukraine-prosecutor-general-4-04-22-transcript'
headers = {"User-agent": 'rsantayana'} 

res = requests.get(url, headers=headers)
if res.status_code == 200:
    subreddit_json = res.raw